In [1]:
import json
from datetime import datetime
from pprint import pprint

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [2]:
from heart import *
from hotel import *
from airline import *

data_and_transformers = [
    ('heart', load_heart(), transformers_heart()),
    ('airline', load_airline(), transformers_airline()),
    ('hotel', load_hotel(), transformers_hotel())
]

In [3]:
models_and_param_grids = [
    ('nb', BernoulliNB(binarize=False), {
        'bernoullinb__alpha': [1.0,]
    }),
    ('xgb', XGBClassifier(random_state=0, objective='binary:logistic'), {
        'xgbclassifier__n_estimators': [100,],
    }),
    ('rf', RandomForestClassifier(random_state=0), {
        'randomforestclassifier__n_estimators': [10,],
    }),
    ('logreg', LogisticRegression(random_state=0), {
        'logisticregression__C': [1,], 
    }),
    ('knn', KNeighborsClassifier(), {
        'kneighborsclassifier__n_neighbors': [5,],
        'kneighborsclassifier__weights': ['distance'],
        # 'kneighborsclassifier__metric': ['hamming'],
    }),
    ('catboost', CatBoostClassifier(verbose=0, random_state=0), {
        'catboostclassifier__n_estimators': [100,],
    }),
]

In [4]:
results = []

for data_name, data, transformers in tqdm(data_and_transformers):
    for model_name, model, param_grid in tqdm(models_and_param_grids):

        pipeline = make_pipeline(*transformers, model)
        grid_search = GridSearchCV(
            estimator=pipeline,
            cv=StratifiedKFold(n_splits=5),
            param_grid=param_grid,
            n_jobs=3,
            verbose=100,
            scoring=['accuracy', 'f1', 'precision', 'recall'],
            refit='f1',
            error_score='raise'
        )
        display(grid_search)

        X_train, X_test, y_train, y_test = data
        grid_search.fit(X=X_train, y=y_train)
        y_pred = grid_search.predict(X_test)

        res = {
            'dataset': data_name,
            'model': model_name,
            'f1': f1_score(y_test, y_pred),
            'accuracy': accuracy_score(y_test, y_pred)
        }
        pprint(res)
        with open(f'{datetime.now()}_{model_name}_{data_name}.json', 'wt') as f:
            json.dump(res, f)
        results.append(res)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['Diabetes']),
                                                                        ('kbinsdiscretizer',
                                                                         KBinsDiscretizer(encode='onehot-dense',
                                                                                          random_state=0,
                                                                                          strategy='uniform',
                                                                                          subsample=None),
                                                                         ['Age',
                                                                          'Education',
                                                                          'Income',
                                                                          'BMI',
                                                                          'MentHlth',
                                                                          'PhysHlth',
                                                                          'GenHlth'])])),
                                       ('bernoullinb',
                                        BernoulliNB(binarize=False))]),
             n_jobs=3, param_grid={'bernoullinb__alpha': [1.0]}, refit='f1',
             scoring=['accuracy', 'f1', 'precision', 'recall'], verbose=100)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START bernoullinb__alpha=1.0......................................
[CV 3/5; 1/1] START bernoullinb__alpha=1.0......................................
[CV 2/5; 1/1] START bernoullinb__alpha=1.0......................................
[CV 3/5; 1/1] END bernoullinb__alpha=1.0; accuracy: (test=0.833) f1: (test=0.369) precision: (test=0.287) recall: (test=0.517) total time=   2.2s
[CV 2/5; 1/1] END bernoullinb__alpha=1.0; accuracy: (test=0.835) f1: (test=0.370) precision: (test=0.289) recall: (test=0.513) total time=   2.1s
[CV 1/5; 1/1] END bernoullinb__alpha=1.0; accuracy: (test=0.833) f1: (test=0.370) precision: (test=0.287) recall: (test=0.521) total time=   2.3s
[CV 4/5; 1/1] START bernoullinb__alpha=1.0......................................
[CV 5/5; 1/1] START bernoullinb__alpha=1.0......................................
[CV 5/5; 1/1] END bernoullinb__alpha=1.0; accuracy: (test=0.834) f1: (test=0.374) precision: (test

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['Diabetes']),
                                                                        ('kbinsdiscretizer',
                                                                         KBinsDiscretizer(encode='onehot-dense',
                                                                                          random_state=...
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=0, ...))]),
             n_jobs=3, param_grid={'xgbclassifier__n_estimators': [100]},
             refit='f1', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=100)

...

[CV 4/5; 1/1] END kneighborsclassifier__n_neighbors=5, kneighborsclassifier__weights=distance; accuracy: (test=0.767) f1: (test=0.663) precision: (test=0.713) recall: (test=0.619) total time=  21.8s
[CV 5/5; 1/1] END kneighborsclassifier__n_neighbors=5, kneighborsclassifier__weights=distance; accuracy: (test=0.773) f1: (test=0.669) precision: (test=0.723) recall: (test=0.623) total time=  21.3s
{'accuracy': 0.7713376329675852,
 'dataset': 'hotel',
 'f1': 0.6741076757789184,
 'model': 'knn'}


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise',
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('simpleimputer',
                                                                         SimpleImputer(fill_value=0,
                                                                                       strategy='constant'),
                                                                         ['children'])],
                                                          verbose_feature_names_out=False)),
                                       ('columntransformer-2',
                                        ColumnTransf...
                                                                          'stays_in_weekend_nights',
                                                                          'hotel',
                                                                          'country',
                                                                          'distribution_channel',
                                                                          'is_repeated_guest',
                                                                          'previous_bookings_not_canceled',
                                                                          'previous_cancellations'])])),
                                       ('catboostclassifier',
                                        <catboost.core.CatBoostClassifier object at 0x7f93f53f0340>)]),
             n_jobs=3, param_grid={'catboostclassifier__n_estimators': [100]},
             refit='f1', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=100)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START catboostclassifier__n_estimators=100........................
[CV 2/5; 1/1] START catboostclassifier__n_estimators=100........................
[CV 3/5; 1/1] START catboostclassifier__n_estimators=100........................
[CV 1/5; 1/1] END catboostclassifier__n_estimators=100; accuracy: (test=0.785) f1: (test=0.644) precision: (test=0.830) recall: (test=0.526) total time=   9.5s
[CV 2/5; 1/1] END catboostclassifier__n_estimators=100; accuracy: (test=0.789) f1: (test=0.652) precision: (test=0.832) recall: (test=0.536) total time=   9.3s
[CV 4/5; 1/1] START catboostclassifier__n_estimators=100........................
[CV 3/5; 1/1] END catboostclassifier__n_estimators=100; accuracy: (test=0.785) f1: (test=0.643) precision: (test=0.830) recall: (test=0.525) total time=   9.4s
[CV 5/5; 1/1] START catboostclassifier__n_estimators=100........................
[CV 4/5; 1/1] END catboostclassifier__n_estimators=100; 

/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV 5/5; 1/1] END catboostclassifier__n_estimators=100; accuracy: (test=0.786) f1: (test=0.643) precision: (test=0.841) recall: (test=0.520) total time=   6.3s
{'accuracy': 0.781221207806349,
 'dataset': 'hotel',
 'f1': 0.6427301326767885,
 'model': 'catboost'}


In [9]:
df_results = pd.DataFrame(results).pivot(
    index='model',
    columns='dataset',
    values=['f1', 'accuracy'],
).swaplevel(axis=1).sort_index(axis=1)
df_results.style.highlight_max().format(precision=3)

In [12]:
df_results.style.highlight_max(props='textbf:--rwrap;').format(precision=3).to_latex('table.tex')